In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
from keras.datasets import boston_housing

In [2]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
X = np.hstack([np.ones([404,1]),x_train])
y = np.reshape(y_train,[-1,1])
Xtest = np.hstack([np.ones([102,1]),x_test])
ytest = np.reshape(y_test,[-1,1])

In [3]:
#Simulate the case that clients hold data separately
X1 = X[0:101]
X2 = X[101:202]
X3 = X[202:303]
X4 = X[303:404]
y1 = y[0:101]
y2 = y[101:202]
y3 = y[202:303]
y4 = y[303:404]

In [4]:
#The following process is completed on server.
##################################################################################################
w1 = 0
w2 = 0.6
max_epoch = 1000000
p = 13                          #Number of features
alpha = 100                     #Initial stride
S = 30                          #Population of Swarms
theta = np.random.rand(p+1,S)   #Initialization of Swarms
v = np.zeros(theta.shape)       #Initialization of Velocities
failure = 0                     #Initialization of Failure of Update (times of gbest not changing)
minimum_record = []             #Record the evolutionary process
##################################################################################################
    
#The server request l_j from clients
##################################################################################################
l1 = X1.shape[0]
l2 = X2.shape[0]
l3 = X3.shape[0]
l4 = X4.shape[0]
##################################################################################################

#The server computes the overall size of dataset
l = l1 + l2 + l3 + l4

for epoch in tqdm(range(0,max_epoch,1)):
    
    #Send the model to all clients, and clients respond the server with client losses.
    #The server do not touch the raw data.
    #The following process is completed on clients.
    ################################################################
    MSE_old1 = np.sum(np.square(np.matmul(X1,theta) - y1),axis = 0)
    MSE_old2 = np.sum(np.square(np.matmul(X2,theta) - y2),axis = 0)
    MSE_old3 = np.sum(np.square(np.matmul(X3,theta) - y3),axis = 0)
    MSE_old4 = np.sum(np.square(np.matmul(X4,theta) - y4),axis = 0)
    ################################################################
    
    #The following process is completed on server
    ################################################################################
    MSE_old = (MSE_old1 + MSE_old2 + MSE_old3 + MSE_old4) 
    minimum_record.append(MSE_old[np.argmin(MSE_old)]/l)
    
    #Adaptive Learning Rate
    if epoch > 0:
        if minimum_record[-1] < minimum_record[-2]:
            alpha = alpha * 2
        else:
            failure += 1
    
    if failure == 5:
        failure = 0
        alpha = alpha / 2
    
    #Display the results
    if epoch%100 == 0:
        print(alpha)
        print(minimum_record[-1])
            
    gbest = theta[:,[np.argmin(MSE_old)]]
    v = alpha * ( w1* v +  w2 * np.random.rand(theta.shape[0],theta.shape[1]) * (gbest - theta) + (1-w1 -w2) * (np.random.rand(theta.shape[0],theta.shape[1]) * 2-1) )
    temp = theta + v
    ################################################################################
    
    
    #Send the model to all clients, and clients respond the server with errors.
    #The server do not touch the raw data.
    #The following process is completed on clients.    
    ################################################################
    MSE_new1 = np.sum(np.square(np.matmul(X1,temp) - y1),axis = 0)
    MSE_new2 = np.sum(np.square(np.matmul(X2,temp) - y2),axis = 0)
    MSE_new3 = np.sum(np.square(np.matmul(X3,temp) - y3),axis = 0)
    MSE_new4 = np.sum(np.square(np.matmul(X4,temp) - y4),axis = 0)
    ################################################################
    
    #The following process is completed on server
    ################################################################################
    MSE_new = MSE_new1 + MSE_new2 + MSE_new3 + MSE_new4

    for particles in range(0,S,1):
        if MSE_new[particles]  < MSE_old[particles]:
            theta[:,[particles]] = temp[:,[particles]]
    ################################################################################

  0%|                                                                          | 275/1000000 [00:00<06:06, 2730.12it/s]

100
9224.183331073686
1.5625
816.6874706732062
0.048828125
600.0820894677348
0.1953125
533.5479699115842
0.0244140625
485.2671989105405
0.01220703125
474.5210689719329
0.01220703125
459.7831503205444


  0%|                                                                          | 904/1000000 [00:00<06:14, 2668.90it/s]

0.0030517578125
450.5942707992601
0.00152587890625
449.13742406748986
0.006103515625
447.0945557339122
0.0244140625
440.19480145931226
0.0244140625
427.30393496586663
0.0030517578125
423.7678948107207


  0%|▏                                                                        | 1782/1000000 [00:00<05:56, 2802.96it/s]

...

2.384185791015625e-05
26.329859440811152
0.00019073486328125
26.329746147704608
9.5367431640625e-05
26.329523234036458
9.5367431640625e-05
26.3293726531884


100%|██████████████████████████████████████████████████████████████████████▉| 999575/1000000 [07:20<00:00, 2176.06it/s]

9.5367431640625e-05
26.32897109497078
9.5367431640625e-05
26.328769979470476
9.5367431640625e-05
26.32858496496119
2.384185791015625e-05
26.32836321637754
4.76837158203125e-05
26.32819169759589


100%|██████████████████████████████████████████████████████████████████████| 1000000/1000000 [07:20<00:00, 2268.75it/s]

4.76837158203125e-05
26.327877538543433
9.5367431640625e-05
26.32750404563265
0.00019073486328125
26.32712709526971


In [5]:
MSE_test = np.sum(np.square(np.matmul(Xtest,gbest) - ytest),axis = 0)/Xtest.shape[0]

In [6]:
print(MSE_test)

[20.63480708]
